In [1]:
import cv2
import numpy as np

定义展示图片的函数

In [2]:
def show(name, img):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

设置最小的匹配点个数和使用线性搜索还是使用k-d树搜索

In [3]:
MIN = 10
FLANN_INDEX_KDTREE = 0

加载图片，并对其进行剪裁

In [4]:
img1 = cv2.imread('SIFT_zuo.jpg')  # query
img2 = cv2.imread('SIFT_you.jpg')  # train

# 将图像缩小0.5倍
imageA = cv2.resize(img1, (0, 0), fx=0.5, fy=0.5)
imageB = cv2.resize(img2, (0, 0), fx=0.5, fy=0.5)

创建特征检测器SIFT 并提取图像的sift特征点

In [5]:
# 创建sift特征检测器
sift = cv2.SIFT_create()
# 提取图片的sift特征点
kp1, descrip1 = sift.detectAndCompute(imageA, None)
kp2, descrip2 = sift.detectAndCompute(imageB, None)

对两幅图像中的特征点进行特征匹配

In [6]:
# 创建字典
# FLANN_INDEX_KDTREE==0代表线性搜索 1代表k-d树搜索
indexParams = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
# 进行最近邻搜索时 最大的比较次数为50
searchParams = dict(checks=50)
flann = cv2.FlannBasedMatcher(indexParams, searchParams)
# 在两幅图像中进行特征匹配
match = flann.knnMatch(descrip1, descrip2, k=2)

过滤特征点并计算视角变换矩阵

In [7]:
good = []
# 过滤特征点
for i, (m, n) in enumerate(match):
    if m.distance < 0.75 * n.distance:
        good.append(m)

# 当筛选后的匹配对大于10时，计算视角变换矩阵
if len(good) > MIN:
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    ano_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
    # 计算出从图像一到图像二的变换矩阵M和掩码mask
    M, mask = cv2.findHomography(src_pts, ano_pts, cv2.RANSAC, 5.0)
    # 对imageB进行视角变化，并把它扩大为两张图片宽的和
    warpImg = cv2.warpPerspective(imageB, np.linalg.inv(M), (imageA.shape[1] + imageB.shape[1], imageB.shape[0]))

计算重叠区域的左右边界列索引

In [8]:
show('res', warpImg)
rows, cols = imageA.shape[:2]
for col in range(0, cols):
    # 开始重叠的最左端
    if imageA[:, col].any() and warpImg[:, col].any():
        left = col
        break

for col in range(cols - 1, 0, -1):
    # 重叠的最右一列
    if imageA[:, col].any() and warpImg[:, col].any():
        right = col
        break

将两张图片进行加权拼接

In [9]:
# 加权处理
res = np.zeros([rows, cols, 3], np.uint8)
for row in range(0, rows):
    for col in range(0, cols):
        if not imageA[row, col].any():  # 如果没有原图，用旋转的填充
            res[row, col] = warpImg[row, col]
        elif not warpImg[row, col].any():
            res[row, col] = imageA[row, col]
        # 计算出两个图像在该列上的重叠长度，按照重叠长度比例对两个像素进行混合
        else:
            srcImgLen = float(abs(col - left))
            testImgLen = float(abs(col - right))
            alpha = srcImgLen / (srcImgLen + testImgLen)
            res[row, col] = np.clip(imageA[row, col] * (1 - alpha) + warpImg[row, col] * alpha, 0, 255)

最后展示结果并保存图片

In [10]:
warpImg[0:imageA.shape[0], 0:imageA.shape[1]] = res
show('res', warpImg)
cv2.imwrite('task2.jpg', warpImg)

True